In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np
import re
import pprint
from collections import OrderedDict

In [3]:
! head ftes_lorvec.out

#RPA-results:
#Nucleus: SN 146
#Excitation: 1 -
#Total number of pairs: 2188
#Number of ph-/ah-pairs: 1554/634
#Maximal excitation-energy for particle-hole-pairs: 200
#Maximal excitation-energy for antiparticle-hole-pairs: 2000
#2 imaginary eigenvalues
#Listing imaginary eigenvalues:
#Imaginary part: 4.383495e-01


In [4]:
146 - 50 # N

96

In [5]:
# 'energy', 'transition_strength'
df_lorvec = pd.read_csv('ftes_lorvec.out', delim_whitespace=True, comment='#', skip_blank_lines=True,
            header=None, names=['U', 'fE1'])

In [6]:
df_lorvec = df_lorvec[(df_lorvec.U >= 0.1) & (df_lorvec.U <= 30)] # MeV

In [7]:
df_lorvec.head()

,U,fE1
10,0.10,0.135900
11,0.11,0.136607
12,0.12,0.137321
13,0.13,0.138042
14,0.14,0.138770


In [8]:
every_10 = df_lorvec.iloc[::10, :]
every_10.reset_index(drop=True, inplace=True)
every_10_T = every_10.T

In [9]:
df = every_10_T
df2 = pd.concat([df], keys=[146], names=['A'])
df3 = pd.concat([df2], keys=[50], names=['Z'])
lorvec_df = df3

In [10]:
stocks = pd.read_csv('http://bit.ly/smallstocks')

In [11]:
stocks.groupby('Symbol').Close.mean()

Symbol
AAPL    112.856667
CSCO     31.480000
MSFT     57.433333
Name: Close, dtype: float64

In [12]:
ser = stocks.groupby(['Symbol', 'Date']).Close.mean()

In [13]:
ser.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['Symbol', 'Date'])

In [14]:
ser.unstack()

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


In [15]:
df = stocks.pivot_table(values='Close', index='Symbol', columns='Date')

In [16]:
stocks.set_index(['Symbol', 'Date'], inplace=True)

In [17]:
ser.loc[:, '2016-10-03']

Symbol
AAPL    112.52
CSCO     31.50
MSFT     57.42
Name: Close, dtype: float64

In [18]:
stocks.index

MultiIndex(levels=[['AAPL', 'CSCO', 'MSFT'], ['2016-10-03', '2016-10-04', '2016-10-05']],
           labels=[[1, 0, 2, 0, 2, 1, 2, 1, 0], [0, 0, 0, 1, 1, 1, 2, 2, 2]],
           names=['Symbol', 'Date'])

In [19]:
stocks.sort_index(inplace=True)

In [20]:
def to_float(iterable):
    return (float(val) for val in iterable)

In [21]:
class ConfigurationSyntaxError(Exception):
    pass

In [22]:
# @TODO 
# df_to_dict(df):
    # get dictionary from dataframe X
# @TODO 
# dict_to_fn(ordered_nested_dict, fname):
    # write dict to file 
# @TODO multiply by constant to convert e^2fm^2 to barn

In [53]:
def fn_to_dict(fname):
    z_from_fname = int(''.join(filter(lambda c: not c.isalpha(), fname)))
    with open(fname) as f:
        contents = f.read()
    
    new_line = re.compile('\n[\s\r]+?\n')
    blocks = new_line.split(contents)
    
    assert blocks[-1].strip() == '', "Last line not empty!"
    assert len(blocks[:-1]) == 82, "Not right number of blocks!"
    # gap from A = 170 to A = 178
    
    mydict = OrderedDict()
    mydict[z_from_fname] =  OrderedDict()
    
    minA, maxA = 400, 0
    for blk in blocks[:-1]: # except last line
        block = blk.splitlines()
    
        nucleus_header = block[0].split()
        Z, A = (int(val) for val in nucleus_header[1::2])
        if (nucleus_header[0::2] != ['Z=', 'A=']) or (Z != z_from_fname):
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)
        if int(A) < minA:
            minA = int(A)
        if int(A) > maxA:
            maxA = int(A)
        
        col_header = block[1].split()
        if col_header[:2] != ['U[MeV]', 'fE1[mb/MeV]']:
            raise ConfigurationSyntaxError('Wrong header inside %s!' % fname)

        columns = (row.split() for row in block[2:])
        c1_vals, c2_vals = zip(*columns)

        d = OrderedDict()
        d[A] = OrderedDict(U=tuple(to_float(c1_vals)),
                         fE1=tuple(to_float(c2_vals)))
        mydict[Z].update(d)
        
    print(f"Amax - Amin = {maxA - minA}, {len(blocks[:-1])} blocks")
    return mydict, minA, maxA

In [117]:
def dict_to_fn(ordered_nested_dict, fname):
    ond = ordered_nested_dict

    assert len(ond.keys()) == 1, "More than one element present in dict!"

    z_from_fname = int(''.join(filter(lambda c: not c.isalpha(), fname)))
    assert list(ond.keys())[0] == z_from_fname, "Elements from dict and file do not match!"

    col_header = f'  U[MeV]  fE1[mb/MeV]'
    
    atomic_mass_numbers = []
    with open(fname, "w") as f:
        for Z in ond.keys():
            for A in ond[Z].keys():
                atomic_mass_numbers.append(A)
                nucleus_header = f' Z={Z:4d} A={A:4d}'
                f.write(f'{nucleus_header}\n')
                f.write(f'{col_header}\n')
                for U, fE1 in zip(ond[Z][A]['U'], ond[Z][A]['fE1']):
                    f.write('{:9.3f}   {:.3E}\n'.format(U, fE1))
                f.write(' \n')
    return atomic_mass_numbers

In [128]:
md, a, b = fn_to_dict('z050')
print(f"Amin = {a}, Amax = {b}")

Amax - Amin = 88, 82 blocks
Amin = 90, Amax = 178


In [129]:
mnum = dict_to_fn(md, 'newz050')

In [130]:
df = dict_to_df(md)

In [137]:
def df_to_dict(df):
    ond = OrderedDict() # ordered_nested_dict
    for Z, A, data_col in df.columns:
        if Z not in ond.keys():
            ond[Z] = OrderedDict()
        if A not in ond[Z].keys():
            ond[Z][A] = OrderedDict()
        ond[Z][A][data_col] = tuple(df[Z][A][data_col].values)
    return ond

In [138]:
md2 = df_to_dict(df)

In [106]:
for i, A in enumerate(mnum[1:], 1):
    if A - mnum[i-1] != 1:
        print(A, A - mnum[i-1])

178 8


In [28]:
md_tst = {'50': {'100': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (0.1359,
                              0.14329,
                              0.151455,
                              0.160519)},
             '101': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (6.510E-03,
                              7.011E-03,
                              7.553E-03,
                              8.134E-03)}
            }
     }

In [127]:
def dict_to_df(ordered_nested_dict):
    reform = {(firstKey, secondKey, thirdKey): values for firstKey, middleDict in ordered_nested_dict.items() for secondKey, innerdict in middleDict.items() for thirdKey, values in innerdict.items()}
    df = pd.DataFrame(reform)
    df = df.T # transpose
    df.index.names=['Z', 'A', None]
    return df.T

In [30]:
# pprint.pprint(md)

In [31]:
df = dict_to_df(md)
df.head()

0         1         2         3         4         5    \
Z  A                                                                    
50 90 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   
      fE1  0.006291  0.006776  0.007300  0.007862  0.008465  0.009107   
   91 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   
      fE1  0.006350  0.006840  0.007369  0.007937  0.008545  0.009193   
   92 U    0.100000  0.200000  0.300000  0.400000  0.500000  0.600000   

                6        7        8        9     ...         290      291  \
Z  A                                             ...                        
50 90 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   
      fE1  0.009790  0.01051  0.01128  0.01209   ...      0.9121   0.9004   
   91 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   
      fE1  0.009882  0.01061  0.01138  0.01220   ...      0.9619   0.9492   
   92 U    0.700000  0.80000  0.90000  1.00000   ...     29.1000  29.2000   

               292      293      294      295      296      297      298  \
Z  A                                                                       
50 90 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   
      fE1   0.8889   0.8775   0.8662   0.8550   0.8439   0.8329   0.8220   
   91 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   
      fE1   0.9365   0.9238   0.9111   0.8984   0.8858   0.8731   0.8605   
   92 U    29.3000  29.4000  29.5000  29.6000  29.7000  29.8000  29.9000   

               299  
Z  A                
50 90 U    30.0000  
      fE1   0.8110  
   91 U    30.0000  
      fE1   0.8479  
   92 U    30.0000  

[5 rows x 300 columns]

In [32]:
df.loc[(50, 146, ['U', 'fE1']), :]

0        1        2        3        4        5       6    \
Z  A                                                                       
50 146 U    0.10000  0.20000  0.30000  0.40000  0.50000  0.60000  0.7000   
       fE1  0.01944  0.02095  0.02259  0.02436  0.02627  0.02831  0.0305   

                7        8        9     ...       290     291     292     293  \
Z  A                                    ...                                     
50 146 U    0.80000  0.90000  1.00000   ...    29.100  29.200  29.300  29.400   
       fE1  0.03284  0.03533  0.03799   ...     1.277   1.266   1.256   1.245   

               294     295     296     297    298     299  
Z  A                                                       
50 146 U    29.500  29.600  29.700  29.800  29.90  30.000  
       fE1   1.234   1.224   1.213   1.201   1.19   1.179  

[2 rows x 300 columns]

In [33]:
lorvec_df.loc[(50, 146, ['U', 'fE1']), :]

0        1         2         3         4         5         6    \
Z  A                                                                            
50 146 U    0.1000  0.20000  0.300000  0.400000  0.500000  0.600000  0.700000   
       fE1  0.1359  0.14329  0.151455  0.160519  0.170631  0.181975  0.194774   

                 7         8         9      ...            290        291  \
Z  A                                        ...                             
50 146 U    0.800000  0.900000  1.000000    ...      29.100000  29.200000   
       fE1  0.209311  0.225938  0.245101    ...       0.090461   0.093698   

                 292        293        294        295        296        297  \
Z  A                                                                          
50 146 U    29.30000  29.400000  29.500000  29.600000  29.700000  29.800000   
       fE1   0.09674   0.099111   0.100767   0.102081   0.103364   0.104466   

                  298        299  
Z  A                              
50 146 U    29.900000  30.000000  
       fE1   0.104862   0.104155  

[2 rows x 300 columns]

In [34]:
df.loc[(50, 146, ['U', 'fE1']), :] = lorvec_df.loc[(50, 146, ['U', 'fE1']), :]

In [35]:
df.loc[(50, 146, ['U', 'fE1']), :]

0        1         2         3         4         5         6    \
Z  A                                                                            
50 146 U    0.1000  0.20000  0.300000  0.400000  0.500000  0.600000  0.700000   
       fE1  0.1359  0.14329  0.151455  0.160519  0.170631  0.181975  0.194774   

                 7         8         9      ...            290        291  \
Z  A                                        ...                             
50 146 U    0.800000  0.900000  1.000000    ...      29.100000  29.200000   
       fE1  0.209311  0.225938  0.245101    ...       0.090461   0.093698   

                 292        293        294        295        296        297  \
Z  A                                                                          
50 146 U    29.30000  29.400000  29.500000  29.600000  29.700000  29.800000   
       fE1   0.09674   0.099111   0.100767   0.102081   0.103364   0.104466   

                  298        299  
Z  A                              
50 146 U    29.900000  30.000000  
       fE1   0.104862   0.104155  

[2 rows x 300 columns]

In [36]:
md_tst = {'50': {'100': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (0.1359,
                              0.14329,
                              0.151455,
                              0.160519)},
             '101': {'col1': (0.100,
                              0.200,
                              0.300,
                              0.400),
                     'col2': (6.510E-03,
                              7.011E-03,
                              7.553E-03,
                              8.134E-03)}
            }
     }

In [37]:
df_tst = dict_to_df(md_tst)
df_tst

0         1         2         3
Z  A                                              
50 100 col1  0.10000  0.200000  0.300000  0.400000
       col2  0.13590  0.143290  0.151455  0.160519
   101 col1  0.10000  0.200000  0.300000  0.400000
       col2  0.00651  0.007011  0.007553  0.008134

In [38]:
df_tst = df_tst.T
pprint.pprint(df_tst.to_dict('list')) # dict is OK
df_tst = df_tst.T
print()
print(df_tst.index) # index is OK

{('50', '100', 'col1'): [0.1, 0.2, 0.3, 0.4],
 ('50', '100', 'col2'): [0.1359, 0.14329, 0.151455, 0.160519],
 ('50', '101', 'col1'): [0.1, 0.2, 0.3, 0.4],
 ('50', '101', 'col2'): [0.00651, 0.007011, 0.007553, 0.008134]}

MultiIndex(levels=[['50'], ['100', '101'], ['col1', 'col2']],
           labels=[[0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 0, 1]],
           names=['Z', 'A', None])


In [39]:
table = df_tst.loc[('50', '100', ['col1', 'col2']), :]
table

0        1         2         3
Z  A                                            
50 100 col1  0.1000  0.20000  0.300000  0.400000
       col2  0.1359  0.14329  0.151455  0.160519

In [40]:
s = df_tst.loc[('50', '100', 'col1'), :]
s

0    0.1
1    0.2
2    0.3
3    0.4
Name: (50, 100, col1), dtype: float64

In [41]:
stocks

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
       2016-10-05  113.05  21453100
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
       2016-10-05   31.59  11808600
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900
       2016-10-05   57.64  16726400

In [42]:
stocks.loc[('AAPL', '2016-10-03'), :] # rows, columns
stocks.loc[('AAPL', '2016-10-03'), 'Close']
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), :]
stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), 'Close']
stocks.loc[('AAPL', ['2016-10-03', '2016-10-04']), 'Close']
stocks.loc[(slice(None), ['2016-10-03', '2016-10-04']), :]

Close    Volume
Symbol Date                        
AAPL   2016-10-03  112.52  21701800
       2016-10-04  113.00  29736800
CSCO   2016-10-03   31.50  14070500
       2016-10-04   31.35  18460400
MSFT   2016-10-03   57.42  19189500
       2016-10-04   57.24  20085900